
### Data extract and upload ( target checkpoints):

In [ ]:
### extract datasets of cifar 10 
import sagemaker
from sagemaker.processing import ScriptProcessor

sagemaker_role = "YOUR_SAGEMAKER_ROLE"#CHANGE THIS ACCORDINGLY
s3_bucket = "YOUR_BUCKET_NAME"#CHANGE THIS ACCORDINGLY
s3_input_prefix = "datasets/"#CHANGE THIS ACCORDINGLY
s3_output_prefix = "extracted_tissue_mnist/dataset/"#CHANGE THIS ACCORDINGLY

script_processor = ScriptProcessor(
    image_uri="763104351884.dkr.ecr.eu-central-1.amazonaws.com/pytorch-training:1.10.2-cpu-py38",  # Use eu-central-1
    role=sagemaker_role,
    instance_count=1,
    instance_type="ml.m5.12xlarge",
    max_runtime_in_seconds=3600*4,
    command=["python3"]  # Ensure Python is used to run the script
)



script_processor.run(
    code="extract_and_upload.py",
    arguments=[
        "--bucket", s3_bucket,
        "--input_s3", s3_input_prefix,
        "--output_s3", s3_output_prefix
    ],
    logs=True,
    wait=True,
)




INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:sagemaker:Creating processing-job with name pytorch-training-2025-03-06-22-45-45-224


............Found 3 .tar.gz files to process.
['datasets/datasets.tar.gz', 'datasets/datasets_cifar_5.tar.gz', 'datasets/datasets_tissuemnist_5.tar.gz']
Processing datasets/datasets_tissuemnist_5.tar.gz...
Uploading extracted_tissue_mnist/dataset//datasets/uniform/tissuemnist/TISSUEMNIST_5_dataset_pointers...
Successfully uploaded extracted_tissue_mnist/dataset//datasets/uniform/tissuemnist/TISSUEMNIST_5_dataset_pointers using multipart upload.
Uploading extracted_tissue_mnist/dataset//datasets/uniform/tissuemnist/dataset_generation.py...
Successfully uploaded extracted_tissue_mnist/dataset//datasets/uniform/tissuemnist/dataset_generation.py using multipart upload.
Uploading extracted_tissue_mnist/dataset//datasets/uniform/tissuemnist/TISSUEMNIST_5_dataset_blueprint.csv...
Successfully uploaded extracted_tissue_mnist/dataset//datasets/uniform/tissuemnist/TISSUEMNIST_5_dataset_blueprint.csv using multipart upload.
Uploading extracted_tissue_mnist/dataset//datasets/uniform/tissuemnist/TI

### launch attack eval : Uniform without DP 


In [ ]:
import boto3
import sagemaker
from sagemaker.processing import ScriptProcessor
from sagemaker import get_execution_role
from sagemaker import image_uris

role = "YOUR_SAGEMAKER_ROLE"#CHANGE THIS ACCORDINGLY
region = "eu-central-1"
sagemaker_session = sagemaker.Session()

# Corrected S3 paths
s3_target_model_path = "s3://YOUR_BUCKET_NAME/extracted_cifar10//uniform/cifar10-5/withoutDP_uniform_cifar10-5/models/"  # The root folder containing 'nodes/' and 'orchestrator/' #CHANGE THIS ACCORDINGLY
s3_attack_model_path = "s3://YOUR_BUCKET_NAME/attack_models/"  # Bucket containing attack models#CHANGE THIS ACCORDINGLY
s3_datasets_path = "extracted_cifar10/dataset//datasets/uniform/cifar10-5/CIFAR10_5_dataset/" #CHANGE THIS ACCORDINGLY
target_models_bucket = "YOUR_BUCKET_NAME"#CHANGE THIS ACCORDINGLY
target_models_prefix = "extracted_cifar10//uniform/cifar10-5/withoutDP_uniform_cifar10-5/models/" #CHANGE THIS ACCORDINGLY
# Corrected output path
output_data_uri = f"{s3_target_model_path}/attack_results/" #CHANGE THIS ACCORDINGLY


framework_version = "4.26.0"  # Use a supported version
pytorch_image_uri = image_uris.retrieve(
    framework="huggingface",
    region="eu-central-1",
    version=framework_version,
    image_scope="training",
    base_framework_version="pytorch1.13.1"
)

print(f"Using Hugging Face PyTorch Image: {pytorch_image_uri}")
processor = ScriptProcessor(
    image_uri=pytorch_image_uri,
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    command=["python3"],
)

processor.run(
    code="launch_attack_cifar10.py",
    arguments=[
        "--attack_bucket", "YOUR_BUCKET_NAME",#CHANGE THIS ACCORDINGLY
        "--attack_model_prefix", "attack_models",#CHANGE THIS ACCORDINGLY
        "--dataset_bucket",  "YOUR_BUCKET_NAME",#CHANGE THIS ACCORDINGLY
        "--dataset_prefix", s3_datasets_path,
        "--batch_size", str(32),
        "--target_models_prefix", target_models_prefix,
        "--target_models_bucket", target_models_bucket,
        "--results_bucket", "YOUR_BUCKET_NAME",#CHANGE THIS ACCORDINGL",
        "--results_prefix", "attack_results/uniform/without_DP/cifar10/",#CHANGE THIS ACCORDINGLY
    ],
    outputs=[sagemaker.processing.ProcessingOutput(source="/opt/ml/processing/output", destination=output_data_uri)],
    logs=True,
    wait=True,
)

print("Processing job launched successfully!")


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:sagemaker.image_uris:Defaulting to only available Python version: py39
INFO:sagemaker.image_uris:Defaulting to only supported image scope: gpu.
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


Using Hugging Face PyTorch Image: 763104351884.dkr.ecr.eu-central-1.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04


INFO:sagemaker:Creating processing-job with name huggingface-pytorch-training-2025-03-07-11-53-48-249


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_0.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_1.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_2.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_3.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_4.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_5.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_6.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_7.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_8.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_9.pth
Processing extract


## Lightly  skew without DP cifar


In [ ]:
import boto3
import sagemaker
from sagemaker.processing import ScriptProcessor
from sagemaker import get_execution_role
from sagemaker import image_uris

role = "YOUR_SAGEMAKER_ROLE"#CHANGE THIS ACCORDINGLY
region = "eu-central-1"
sagemaker_session = sagemaker.Session()

# Corrected S3 paths
s3_target_model_path = "s3://YOUR_BUCKET_NAME/extracted_cifar10//lightly_skewed/cifar10-5/withoutDP_ls_cifar10-5/models/"  # The root folder containing 'nodes/' and 'orchestrator/' #CHANGE THIS ACCORDINGLY
s3_attack_model_path = "s3://YOUR_BUCKET_NAME/attack_models/"  # Bucket containing attack models#CHANGE THIS ACCORDINGLY
s3_datasets_path = "extracted_cifar10/dataset//datasets/lightly_skewed/cifar10-5/CIFAR10_5_dataset/" #CHANGE THIS ACCORDINGLY 
target_models_bucket = "YOUR_BUCKET_NAME"#CHANGE THIS ACCORDINGLY
target_models_prefix = "extracted_cifar10//lightly_skewed/cifar10-5/withoutDP_ls_cifar10-5/models/" #CHANGE THIS ACCORDINGLY
# Corrected output path
output_data_uri = f"{s3_target_model_path}/attack_results_cifar10_ls_WithoutDP/"#CHANGE THIS ACCORDINGLY


framework_version = "4.26.0"  # Use a supported version
pytorch_image_uri = image_uris.retrieve(
    framework="huggingface",
    region="eu-central-1",
    version=framework_version,
    image_scope="training",
    base_framework_version="pytorch1.13.1"
)

print(f"Using Hugging Face PyTorch Image: {pytorch_image_uri}")
processor = ScriptProcessor(
    image_uri=pytorch_image_uri,
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    command=["python3"],
)

processor.run(
    code="launch_attack_cifar10.py",
    arguments=[
        "--attack_bucket", "YOUR_BUCKET_NAME",#CHANGE THIS ACCORDINGLY
        "--attack_model_prefix", "attack_models",#CHANGE THIS ACCORDINGLY
        "--dataset_bucket",  "YOUR_BUCKET_NAME", #CHANGE THIS ACCORDINGLY
        "--dataset_prefix", s3_datasets_path,
        "--batch_size", str(32),
        "--target_models_prefix", target_models_prefix,
        "--target_models_bucket", target_models_bucket,
        "--results_bucket", "YOUR_BUCKET_NAME",#CHANGE THIS ACCORDINGLY
        "--results_prefix", "attack_results/ls/without_DP/cifar10/",#CHANGE THIS ACCORDINGLY
    ],
    outputs=[sagemaker.processing.ProcessingOutput(source="/opt/ml/processing/output", destination=output_data_uri)],
    logs=True,
    wait=True,
)

print("Processing job launched successfully!")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Using Hugging Face PyTorch Image: 763104351884.dkr.ecr.eu-central-1.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04


INFO:sagemaker:Creating processing-job with name huggingface-pytorch-training-2025-03-07-15-41-19-378


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_0.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_1.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_2.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_3.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_4.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_5.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_6.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_7.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_8.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_9.pth
Processing extract

## Highly skewed without DP cifar 

In [ ]:
import boto3
import sagemaker
from sagemaker.processing import ScriptProcessor
from sagemaker import get_execution_role
from sagemaker import image_uris

role = "YOUR_SAGEMAKER_ROLE"#CHANGE THIS ACCORDINGLY
region = "eu-central-1"
sagemaker_session = sagemaker.Session()

# Corrected S3 paths
s3_target_model_path = "s3://YOUR_BUCKET_NAME/extracted_cifar10//highly_skewed/cifar10-5/withoutDP_hs_cifar10-5/models/"  # The root folder containing 'nodes/' and 'orchestrator/' #CHANGE THIS ACCORDINGLY
s3_attack_model_path = "s3://YOUR_BUCKET_NAME/attack_models/"  # Bucket containing attack models#CHANGE THIS ACCORDINGLY
s3_datasets_path = "extracted_cifar10/dataset//datasets/highly_skewed/cifar10-5/CIFAR10_5_dataset/"#CHANGE THIS ACCORDINGLY

target_models_bucket = "YOUR_BUCKET_NAME"#CHANGE THIS ACCORDINGLY
target_models_prefix = "extracted_cifar10//highly_skewed/cifar10-5/withoutDP_hs_cifar10-5/models/" #CHANGE THIS ACCORDINGLY
# Corrected output path
output_data_uri = f"{s3_target_model_path}/attack_results_cifar10_hs_WithoutDP/"#CHANGE THIS ACCORDINGLY

framework_version = "4.26.0"  # Use a supported version
pytorch_image_uri = image_uris.retrieve(
    framework="huggingface",
    region="eu-central-1",
    version=framework_version,
    image_scope="training",
    base_framework_version="pytorch1.13.1"
)

print(f"Using Hugging Face PyTorch Image: {pytorch_image_uri}")
processor = ScriptProcessor(
    image_uri=pytorch_image_uri,
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    command=["python3"],
)

processor.run(
    code="launch_attack_cifar10.py",
    arguments=[
        "--attack_bucket", "YOUR_BUCKET_NAME",#CHANGE THIS ACCORDINGLY
        "--attack_model_prefix", "attack_models",#CHANGE THIS ACCORDINGLY
        "--dataset_bucket",  "YOUR_BUCKET_NAME",#CHANGE THIS ACCORDINGLY
        "--dataset_prefix", s3_datasets_path,
        "--batch_size", str(32),
        "--target_models_prefix", target_models_prefix,
        "--target_models_bucket", target_models_bucket,
        "--results_bucket", "YOUR_BUCKET_NAME",#CHANGE THIS ACCORDINGLY
        "--results_prefix", "attack_results/hs/without_DP/cifar10/",#CHANGE THIS ACCORDINGLY
    ],
    outputs=[sagemaker.processing.ProcessingOutput(source="/opt/ml/processing/output", destination=output_data_uri)],
    logs=True,
    wait=True,
)

print("Processing job launched successfully!")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Using Hugging Face PyTorch Image: 763104351884.dkr.ecr.eu-central-1.amazonaws.com/huggingface-pytorch-training:1.13.1-transformers4.26.0-gpu-py39-cu117-ubuntu20.04


INFO:sagemaker:Creating processing-job with name huggingface-pytorch-training-2025-03-07-21-24-34-363


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_0.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_1.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_2.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_3.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_4.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_5.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_6.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_7.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_8.pth
Attempting to download from S3: sagemaker-cifar10-mia/attack_models/attack_model_9.pth
Processing extract